<a href="https://colab.research.google.com/github/mr-nudo/intelligent-tools/blob/master/KNN_DT_RF_XGB_Q5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the libraries

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Setting random seed

In [ ]:
random.seed(25)
np.random.seed(25)


# Load the dataset

In [ ]:
# Load the dataset
data_path = 'path_to_imbalanced_heart_data.csv'
heart_data = pd.read_csv(data_path)


# TODO: Explore the dataset

## TODO: Print basic information

In [ ]:
# TODO: Basic information


## TODO: Print the distribution of target variable

In [ ]:
# TODO: Distribution of the target variable

## TODO: Plot histograms of numerical attributes

In [ ]:
# TODO: Plotting histograms for numerical attributes


## TODO: Visualise the correlation matrix

In [ ]:
# TODO: Correlation matrix


# TODO: Scale the features

In [ ]:
# TODO: Scaling features


# TODO: Create the training and testing datasets

In [ ]:
# TODO:

# TODO: Train all the models

In [ ]:
# TODO: Define parameter sets


# TODO: Classifiers with their parameter sets


# TODO: Evaluate each classifier with each parameter set
